# Importing libraries

In [ ]:
!pip install tensorflow_decision_forests

     |████████████████████████████████| 15.8 MB 4.6 MB/s 
     |████████████████████████████████| 511.7 MB 3.8 kB/s 
     |████████████████████████████████| 5.8 MB 42.4 MB/s 
     |████████████████████████████████| 438 kB 62.5 MB/s 
     |████████████████████████████████| 1.6 MB 34.4 MB/s 
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.8.0
    Uninstalling tensorflow-estimator-2.8.0:
      Successfully uninstalled tensorflow-estimator-2.8.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.8.0
    Uninstalling tensorboard-2.8.0:
      Successfully uninstalled tensorboard-2.8.0
  Attempting uninstall: keras
    Found existing installation: keras 2.8.0
    Uninstalling keras-2.8.0:
      Successfully uninstalled keras-2.8.0
  Attempting uninstall: gast
    Found existing installation: gast 0.5.3
    Uninstalling gast-0.5.3:
      Successfully uninstalled gast-0.5.3
  Attempting uninstall: flatbuffers
 

In [ ]:
!pip install wurlitzer

In [ ]:
import tensorflow_decision_forests as tfdf

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf

# PCA implementation

In [ ]:
def PCATransform(numComponents, df):
  import numpy as np
  # calculating eigenvalues+eigenvectors from the covariance matrix
  df = df - np.mean(df, axis = 0)
  transposedDF = df.transpose()
  covariance = np.cov(transposedDF)
  eigenvalues, eigenvectors = np.linalg.eigh(covariance)

  # sorting in descending order
  sorted_index = np.argsort(eigenvalues)[::-1]
  eigenvalues = eigenvalues[sorted_index]
  eigenvectors = eigenvectors[:,sorted_index]

  # choosing the top numComponents eigenvectors
  principalAxes = eigenvectors[:,0:numComponents]

  # projecting the data onto the principleAxes
  newData = df.dot(principalAxes)

  return newData

# Explained Variance

In [ ]:
def explainedVariance(thresh, df):
  import numpy as np

  # calculating eigenvalues+eigenvectors from the covariance matrix
  df = df - np.mean(df, axis = 0)
  transposedDF = df.transpose()
  covariance = np.cov(transposedDF)
  eigenvalues, eigenvectors = np.linalg.eigh(covariance)

  # sorting in descending order
  sorted_index = np.argsort(eigenvalues)[::-1]
  eigenvalues = eigenvalues[sorted_index]
  eigenvectors = eigenvectors[:,sorted_index]

  # calculating the cumulative sum of the explained variance
  l = []
  i = 0
  csum = 0
  eigensum = np.sum(eigenvalues)
  while csum/eigensum < thresh and i < len(eigenvalues):
    csum += eigenvalues[i]
    l.append(csum/eigensum)
    i += 1
  print(i)
  df = pd.DataFrame(l, columns= ["Explained variance"])
  df.to_excel("eigenpictures.xlsx")
  return l

In [ ]:
def explainedVarianceSVD(thresh, arr):
  import numpy as np

  arr = np.square(arr)
  l = []
  i = 0
  csum = 0
  total = np.sum(arr)

  while csum/total < thresh and i < len(arr):
    csum += arr[i]
    l.append(csum/total)
    i += 1
  print(i)
  return l

# Projections

In [ ]:
def downloadProjection(numComponents, df, index):
  import numpy as np
  from PIL import Image as im
  # calculating eigenvalues+eigenvectors from the covariance matrix
  dfcopy = df.copy()
  df = df - np.mean(df, axis = 0)
  transposedDF = df.transpose()
  covariance = np.cov(transposedDF)
  eigenvalues, eigenvectors = np.linalg.eigh(covariance)

  # sorting in descending order
  sorted_index = np.argsort(eigenvalues)[::-1]
  eigenvalues = eigenvalues[sorted_index]
  eigenvectors = eigenvectors[:,sorted_index]

  # choosing the top numComponents eigenvectors
  principleAxes = eigenvectors[:,0:numComponents]
  newData = df @ principleAxes @ principleAxes.transpose()
  newData = np.array(newData)
  mean = np.resize(np.mean(dfcopy, axis = 0), (9999, 784))
  newData = np.add(newData, mean)

  # setting up the image
  imgArr = newData[index, :]
  imgArr = np.array(imgArr)
  imgArr = np.reshape(imgArr, (28,28))
  imgArr = imgArr.astype(np.uint8)
  
  data = im.fromarray(imgArr)
      
  # saving the final output as a PNG file
  data.save('img' + str(index) + '.png')


In [ ]:
# gets the projection of an image into the eigenspace and back
def getProjection(numComponents, df):
  import numpy as np
  from PIL import Image as im
  # calculating eigenvalues+eigenvectors from the covariance matrix
  dfcopy = df.copy()
  df = df - np.mean(df, axis = 0)
  transposedDF = df.transpose()
  covariance = np.cov(transposedDF)
  eigenvalues, eigenvectors = np.linalg.eigh(covariance)

  # sorting in descending order
  sorted_index = np.argsort(eigenvalues)[::-1]
  eigenvalues = eigenvalues[sorted_index]
  eigenvectors = eigenvectors[:,sorted_index]

  # choosing the top numComponents eigenvectors
  principleAxes = eigenvectors[:,0:numComponents]

  # Projecting into the eigenspace and back
  newData = df @ principleAxes @ principleAxes.transpose()
  newData = np.array(newData)

  # adding the mean back
  mean = np.resize(np.mean(dfcopy, axis = 0), (9999, 784))
  newData = np.add(newData, mean)

  newData = np.reshape(newData, (df.shape[0],28,28))
  return newData

In [ ]:
def downloadImage(df, index):
  import numpy as np
  from PIL import Image as im
  newData = np.array(df)
  # print(newData.shape)
  imgArr = newData[index, :]
  # print(imgArr.shape)
  imgArr = np.array(imgArr)
  imgArr = np.reshape(imgArr, (28,28))
  imgArr = imgArr.astype(np.uint8)
  
  data = im.fromarray(imgArr)
      
  # saving the final output 
  # as a PNG file
  data.save('img.png')

In [ ]:
PCATransform(5, pd.read_csv('heart.csv'))

,0,1,2,3,4
0,12.267726,2.870002,-14.968778,6.893478,0.319963
1,-2.689662,-39.873863,-0.878693,-10.583550,-2.380576
2,42.950640,-23.639530,-1.759624,-7.786466,-0.356521
3,10.945206,-28.441101,7.043844,8.328269,-0.034165
4,-106.978583,-15.879436,14.862019,3.224170,0.291158
...,...,...,...,...,...
298,4.553591,27.494199,-5.374418,-2.981877,1.032489
299,-16.428494,12.925387,25.582195,-9.397658,0.811133
300,51.963290,13.329155,-15.487471,11.637005,-2.532174
301,114.755392,36.439808,-0.127538,0.511101,-0.275364


In [ ]:
# downloadProjection(500, pd.read_csv('sample_data/mnist_test.csv').iloc[:,1:], 0)

In [ ]:
# downloadImage(pd.read_csv('sample_data/mnist_test.csv').iloc[:,1:])

In [ ]:
# for i in range(0, 20):
#  downloadProjection(149, pd.read_csv('sample_data/mnist_test.csv').iloc[:,1:], i)

# MNIST digit classification

In [ ]:
df = pd.read_csv('sample_data/mnist_test.csv')
X = df.iloc[:,1:]
y = df.iloc[:,0:1]

## Percent of explained variance vs. number of eigenvectors (number of features)

In [ ]:
# number of eigenvectors needed to account for 95% of the variance in the data
cumulativeExplainedVariance = explainedVariance(.95, X)

149


## Searching for best parameters (MLP)

In [ ]:
from sklearn.metrics import make_scorer
from sklearn.metrics import accuracy_score
def accuracy_categorical(y, y_pred, **kwargs):
    return accuracy_score(y.argmax(1), y_pred, **kwargs)
our_accuracy = make_scorer(accuracy_categorical)

In [ ]:
# Define training and testing sets
from sklearn.model_selection import train_test_split
from keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD

df = pd.read_csv('sample_data/mnist_test.csv')
X = df.iloc[:,1:]
y = df.iloc[:,0:1]
X = PCATransform(149, X)

# convert from integers to floats
X = X.astype('float32')
# normalize to range 0-1
X = X / 255.0
y = to_categorical(y)

X_trainAll, X_test, y_trainAll, y_test = train_test_split(X, y, test_size = 0.2)
X_train, X_valid, y_train, y_valid = train_test_split(X_trainAll, y_trainAll, test_size = 0.2)

# Define function for neural network model with parameters
def create_model(n_hidden = 1, n_neurons = 100, learning_rate = 0.01, momentum = .9):
    model = Sequential()
    model.add(Dense(149, activation='relu', kernel_initializer='he_uniform'))
    for layer in range(0, n_hidden):
        model.add(Dense(n_neurons, activation = 'relu', kernel_initializer='he_uniform'))
    model.add(Dense(10, activation = 'softmax'))
    # compile model
    opt = SGD(learning_rate=learning_rate, momentum=momentum)
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Define the estimator
estimator = KerasClassifier(build_fn = create_model, epochs = 10, batch_size = 32, verbose = 0)

# Define the search space for the hyperparameters
params = {"n_hidden": [2, 3, 4],
          "n_neurons": [100, 150, 200]}

# Perform a search on different hyperparameters
# Use a randomized search as a grid search would take too long
# X_valid and y_valid are used for early stopping only
# The randomized search uses k-fold cross validation
from sklearn.model_selection import RandomizedSearchCV

search = RandomizedSearchCV(estimator, params, n_iter = 5, scoring = our_accuracy, cv = 5)
search.fit(X_train, y_train, validation_data = (X_valid, y_valid))

# Get the best estimator
print("Best parameters:")
print(search.best_params_)
print(search.best_score_)
model = search.best_estimator_.model

# Test it

_, acc = model.evaluate(X_test, y_test, verbose=0)
print('> %.3f' % (acc * 100.0))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:37: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.


Best parameters:
{'n_neurons': 200, 'n_hidden': 3}
0.9478060252150117
> 95.000


## Varying number of principal components fed into a neural network

In [ ]:
def create_model():
	from tensorflow.keras.models import Sequential
	from tensorflow.keras.layers import Dense
	from tensorflow.keras.optimizers import SGD
	model = Sequential()
	model.add(Dense(149, activation='relu', kernel_initializer='he_uniform'))
	model.add(Dense(200, activation='relu', kernel_initializer='he_uniform'))
	model.add(Dense(200, activation='relu', kernel_initializer='he_uniform'))
	model.add(Dense(10, activation='softmax'))
	# compile model
	opt = SGD(learning_rate=0.01, momentum=0.9)
	model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
	return model
def runMLP():
	from tensorflow.keras.datasets import mnist
	from tensorflow.keras.utils import to_categorical
	from keras.wrappers.scikit_learn import KerasClassifier

	df = pd.read_csv('sample_data/mnist_test.csv')
	X = df.iloc[:,1:]
	y = df.iloc[:,0:1]
	X = PCATransform(149, X)
	
	X = X.astype('float32')
	X = X / 255.0
	y = to_categorical(y)
 
	from sklearn.model_selection import train_test_split
	X_trainAll, X_test, y_trainAll, y_test = train_test_split(X, y, test_size = 0.2)
	X_train, X_valid, y_train, y_valid = train_test_split(X_trainAll, y_trainAll, test_size = 0.2)

	# fit model
	model = create_model()

	# Define the stopping callbacks
	from keras.callbacks import EarlyStopping
	from keras.callbacks import ModelCheckpoint
	es = EarlyStopping(monitor = 'val_loss', mode = 'min', verbose = 1, patience = 50)
	mc = ModelCheckpoint('best_model.tf', monitor = 'val_accuracy', mode = 'max', verbose = 0, save_best_only = True)

	# Train the model
	history = model.fit(X_train, y_train, validation_data = (X_valid, y_valid), epochs = 10, batch_size = 32, verbose = 0, callbacks = [es, mc])

	# save model
	model.save('final_model.h5')
	# evaluate model on test dataset
	_, acc = model.evaluate(X_test, y_test, verbose=0)
	print('> %.3f' % (acc * 100.0))

	# Define the estimator
	from sklearn.model_selection import cross_val_score
	from sklearn.pipeline import Pipeline
	from sklearn.preprocessing import MinMaxScaler

	estimator = KerasClassifier(build_fn = create_model, epochs = 10, batch_size = 32, verbose = 0)

	results = cross_val_score(estimator, X, y, scoring = our_accuracy, cv = 5)
	print("Results:")
	print("Mean:", results.mean())
	print(results)

In [ ]:
runMLP()

INFO:tensorflow:Assets written to: best_model.tf/assets
INFO:tensorflow:Assets written to: best_model.tf/assets
INFO:tensorflow:Assets written to: best_model.tf/assets
INFO:tensorflow:Assets written to: best_model.tf/assets
INFO:tensorflow:Assets written to: best_model.tf/assets
INFO:tensorflow:Assets written to: best_model.tf/assets
INFO:tensorflow:Assets written to: best_model.tf/assets
INFO:tensorflow:Assets written to: best_model.tf/assets
> 95.300


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:55: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.


Results:
Mean: 0.945895847923962
[0.9275     0.9325     0.949      0.962      0.95847924]


## Varying number of principal components fed into a 2d convolutional neural network

In [ ]:
def create_model():
	from tensorflow.keras.models import Sequential
	from tensorflow.keras.layers import Conv2D
	from tensorflow.keras.layers import MaxPooling2D
	from tensorflow.keras.layers import Dense
	from tensorflow.keras.layers import Flatten
	from tensorflow.keras.optimizers import SGD
	model = Sequential()
	model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', input_shape=(n, n, 1)))
	model.add(MaxPooling2D((2, 2)))
	model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform'))
	model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform'))
	model.add(MaxPooling2D((2, 2)))
	model.add(Flatten())
	model.add(Dense(10, activation='softmax'))
	# compile model
	opt = SGD(learning_rate=0.01, momentum=0.9)
	model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

def runCNN(n):
	from tensorflow.keras.datasets import mnist
	from tensorflow.keras.utils import to_categorical

	df = pd.read_csv('sample_data/mnist_test.csv')
	X = df.iloc[:,1:]
	y = df.iloc[:,0:1]
	X = PCATransform(n*n, X)

	X = np.array(X)
	y = to_categorical(y)
 
	from sklearn.model_selection import train_test_split
	trainX, testX, trainY, testY = train_test_split(X, y, test_size = 0.2)

	trainX = trainX.reshape((trainX.shape[0], n, n, 1))
	testX = testX.reshape((testX.shape[0], n, n, 1))
	# one hot encode target values
	trainY = to_categorical(trainY)
	testY = to_categorical(testY)

  # convert from integers to floats
	trainX = trainX.astype('float32')
	testX = testX.astype('float32')
	# normalize to range 0-1
	trainX = trainX / 255.0
	testX = testX / 255.0
 
  # load dataset
	# trainX, trainY, testX, testY = load_dataset()
	# prepare pixel data
	# trainX, testX = prep_pixels(trainX, testX)
	# define model
	# model = define_model()
	# fit model
	model.fit(trainX, trainY, epochs=10, batch_size=32, verbose=0)
	# save model
	model.save('final_model.h5')
	# load dataset
	# trainX, trainY, testX, testY = load_dataset()
	# prepare pixel data
	# trainX, testX = prep_pixels(trainX, testX)
	# evaluate model on test dataset
	_, acc = model.evaluate(testX, testY, verbose=0)
	print('> %.3f' % (acc * 100.0))

In [ ]:
runCNN(14)

> 91.500


## Trying CNN with projected images

In [ ]:
def runCNN2(n):
  from tensorflow.keras.datasets import mnist
  from tensorflow.keras.utils import to_categorical
  from tensorflow.keras.models import Sequential
  from tensorflow.keras.layers import Conv2D
  from tensorflow.keras.layers import MaxPooling2D
  from tensorflow.keras.layers import Dense
  from tensorflow.keras.layers import Flatten
  from tensorflow.keras.optimizers import SGD

  df = pd.read_csv('sample_data/mnist_test.csv')
  X = df.iloc[:,1:]
  y = df.iloc[:,0:1]
  X = getProjection(n, X)


  X = X.astype('float32')
  X = X / 255.0
  y = to_categorical(y)
  from sklearn.model_selection import train_test_split
  trainX, testX, trainY, testY = train_test_split(X, y, test_size = 0.2)

  model = Sequential()
  model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', input_shape=(28, 28, 1)))
  model.add(MaxPooling2D((2, 2)))
  model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform'))
  model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform'))
  model.add(MaxPooling2D((2, 2)))
  model.add(Flatten())
  model.add(Dense(10, activation='softmax'))
  # compile model
  opt = SGD(learning_rate=0.01, momentum=0.9)
  model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

  # load dataset
  # trainX, trainY, testX, testY = load_dataset()
  # prepare pixel data
  # trainX, testX = prep_pixels(trainX, testX)
  # define model
  # model = define_model()
  # fit model
  model.fit(trainX, trainY, epochs=10, batch_size=32, verbose=0)
  # save model
  model.save('final_model.h5')
  # load dataset
  # trainX, trainY, testX, testY = load_dataset()
  # prepare pixel data
  # trainX, testX = prep_pixels(trainX, testX)
  # evaluate model on test dataset
  _, acc = model.evaluate(testX, testY, verbose=0)
  print('> %.3f' % (acc * 100.0))

In [ ]:
runCNN2(323)

> 98.400


## K-nearest neighbors classifier

In [ ]:
# Define training and testing sets
from sklearn.model_selection import train_test_split
from keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD

df = pd.read_csv('sample_data/mnist_test.csv')
X = df.iloc[:,1:]
y = df.iloc[:,0:1]
X = PCATransform(149, X)

# convert from integers to floats
X = X.astype('float32')
# normalize to range 0-1
X = X / 255.0
y = to_categorical(y)

X_trainAll, X_test, y_trainAll, y_test = train_test_split(X, y, test_size = 0.2)
X_train, X_valid, y_train, y_valid = train_test_split(X_trainAll, y_trainAll, test_size = 0.2)

# Define the estimator
from sklearn.neighbors import KNeighborsClassifier
model = KNeighborsClassifier()

# Define the search space for the hyperparameters
params = {"n_neighbors": [5, 11, 13, 15],
          "weights": ['uniform', 'distance'],
          "p": [2, 5]}

# Perform a search on different hyperparameters
# Use a randomized search as a grid search would take too long
# X_valid and y_valid are used for early stopping only
# The randomized search uses k-fold cross validation
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV

search = GridSearchCV(model, params, scoring = 'accuracy', cv = 5)
search.fit(X_train, y_train)

# Get the best estimator
print("Best parameters:")
print(search.best_params_)
print(search.best_score_)
model = search.best_estimator_

Best parameters:
{'n_neighbors': 5, 'p': 5, 'weights': 'distance'}
0.9367101983971853


In [ ]:
def runKNN():
	from tensorflow.keras.datasets import mnist
	from tensorflow.keras.utils import to_categorical
	from keras.wrappers.scikit_learn import KerasClassifier

	df = pd.read_csv('sample_data/mnist_test.csv')
	X = df.iloc[:,1:]
	y = df.iloc[:,0:1]
	X = PCATransform(149, X)
	
	X = X.astype('float32')
	X = X / 255.0
 
	from sklearn.model_selection import train_test_split
	X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)
	
	from sklearn.neighbors import KNeighborsClassifier
	model = KNeighborsClassifier(n_neighbors = 5, p = 5, weights = 'distance')

	from sklearn.model_selection import cross_val_score
	results = cross_val_score(model, X, y, scoring = 'accuracy', cv = 5)
	print("Results:")
	print("Mean:", results.mean())
	print(results)

In [ ]:
runKNN()

/usr/local/lib/python3.7/dist-packages/sklearn/neighbors/_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/usr/local/lib/python3.7/dist-packages/sklearn/neighbors/_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/usr/local/lib/python3.7/dist-packages/sklearn/neighbors/_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/usr/local/lib/python3.7/dist-packages/sklearn/neighbors/_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for exam

Results:
Mean: 0.949396548274137
[0.9355     0.9335     0.946      0.9665     0.96548274]


# Chess piece classification using pre-trained convolutional neural networks

In [ ]:
pip install split-folders

In [ ]:
!wget https://github.com/davidmallasen/LiveChess2FEN/releases/download/v0.1.0/ChessPiecesDataset.zip

!unzip /content/ChessPiecesDataset.zip

Streaming output truncated to the last 5000 lines.
  inflating: ChessPiecesDataset/p/6360.jpg  
  inflating: ChessPiecesDataset/p/6361.jpg  
  inflating: ChessPiecesDataset/p/6362.jpg  
  inflating: ChessPiecesDataset/p/6363.jpg  
  inflating: ChessPiecesDataset/p/6364.jpg  
  inflating: ChessPiecesDataset/p/6365.jpg  
  inflating: ChessPiecesDataset/p/6366.jpg  
  inflating: ChessPiecesDataset/p/6367.jpg  
  inflating: ChessPiecesDataset/p/6368.jpg  
  inflating: ChessPiecesDataset/p/6369.jpg  
  inflating: ChessPiecesDataset/p/637.jpg  
  inflating: ChessPiecesDataset/p/6370.jpg  
  inflating: ChessPiecesDataset/p/6371.jpg  
  inflating: ChessPiecesDataset/p/6372.jpg  
  inflating: ChessPiecesDataset/p/6373.jpg  
  inflating: ChessPiecesDataset/p/6374.jpg  
  inflating: ChessPiecesDataset/p/6375.jpg  
  inflating: ChessPiecesDataset/p/6376.jpg  
  inflating: ChessPiecesDataset/p/6377.jpg  
  inflating: ChessPiecesDataset/p/6378.jpg  
  inflating: ChessPiecesDataset/p/6379.jpg  
  inf

In [ ]:
import splitfolders
splitfolders.ratio('/content/ChessPiecesDataset', output = "output", seed = 1, ratio = (.8, .1, .1))

Copying files: 54794 files [00:17, 3117.71 files/s]


In [ ]:
import os
base_dir = '/content/output/'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'val')
test_dir = os.path.join(base_dir, 'test')

In [ ]:
from tensorflow.keras.optimizers import SGD
batch_size = 256
opt = SGD(learning_rate=0.01, momentum=0.9)

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

# randomly rotate the images to allow the model to learn variations in orientation
datagen = ImageDataGenerator(
        rotation_range=5,
        # width_shift_range=0.1,
        # height_shift_range=0.1,
        rescale=1./255,
        horizontal_flip=True,
        fill_mode='nearest')

test_datagen = ImageDataGenerator(rescale=1./255)

from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten
from keras.models import Model
from keras.applications.imagenet_utils import decode_predictions
from os import path
# Define the stopping callbacks
from keras.callbacks import EarlyStopping
es = EarlyStopping(monitor = 'val_loss', mode = 'min', verbose = 1, patience = 20)

## VGG16

In [ ]:
shape = (224, 224)
train_gen = datagen.flow_from_directory(train_dir, target_size = shape, batch_size = batch_size, class_mode = 'categorical', color_mode = 'rgb', shuffle=True)
valid_gen = test_datagen.flow_from_directory(validation_dir, target_size = shape, batch_size = batch_size, class_mode = 'categorical', color_mode = 'rgb', shuffle=False)
test_gen = test_datagen.flow_from_directory(test_dir, target_size = shape, batch_size = batch_size, class_mode = 'categorical', color_mode = 'rgb', shuffle=False)

from keras.applications.vgg16 import VGG16

base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224,224,3)) 
 
# Freeze convolutional layers
for layer in base_model.layers:
    layer.trainable = False    

# Establish new fully connected block
x = base_model.output
x = Flatten()(x)  # flatten from convolution tensor output  
x = Dense(1000, activation='relu')(x) # number of layers and units are hyperparameters, as usual
predictions = Dense(13, activation='softmax')(x) # should match # of classes predicted

# this is the model we will train
model_VGG16 = Model(inputs=base_model.input, outputs=predictions)
model_VGG16.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

58900480/58889256 [==============================] - 1s 0us/step


In [ ]:
epochs = 20
if os.path.exists('model_VGG16.h5'):
  model_VGG16 = tf.keras.models.load_model('model_VGG16.h5')
  print("Model loaded")

history = model_VGG16.fit(
    train_gen, 
    epochs=epochs,
    verbose = 1,
    validation_data=test_gen,
    callbacks = [es]
    )
model_VGG16.save('model_VGG16.h5') 

NameError: ignored

In [ ]:
model_VGG16 = tf.keras.models.load_model('model_VGG16.h5')
_, acc = model_VGG16.evaluate(valid_gen, verbose=0)
print('> %.3f' % (acc * 100.0))

> 95.232


## InceptionV3

In [ ]:
shape = (150, 150)
train_gen = datagen.flow_from_directory(train_dir, target_size = shape, batch_size = batch_size, class_mode = 'categorical', color_mode = 'rgb', shuffle=True)
valid_gen = test_datagen.flow_from_directory(validation_dir, target_size = shape, batch_size = batch_size, class_mode = 'categorical', color_mode = 'rgb', shuffle=False)
test_gen = test_datagen.flow_from_directory(test_dir, target_size = shape, batch_size = batch_size, class_mode = 'categorical', color_mode = 'rgb', shuffle=False)

from tensorflow.keras.applications.inception_v3 import InceptionV3

base_model = InceptionV3(input_shape = (150, 150, 3), include_top = False, weights = 'imagenet')

# Freeze convolutional layers
for layer in base_model.layers:
    layer.trainable = False    

# Establish new fully connected block
x = base_model.output
x = Flatten()(x)  # flatten from convolution tensor output  
x = Dense(1000, activation='relu')(x) # number of layers and units are hyperparameters, as usual
predictions = Dense(13, activation='softmax')(x) # should match # of classes predicted

# this is the model we will train
model_InceptionV3 = Model(inputs=base_model.input, outputs=predictions)
model_InceptionV3.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

87924736/87910968 [==============================] - 1s 0us/step


In [ ]:
if os.path.exists('model_InceptionV3.h5'):
  model_InceptionV3 = tf.keras.models.load_model('model_InceptionV3.h5')
  print("Model loaded")
epochs = 20

history = model_InceptionV3.fit(
    train_gen, 
    epochs=epochs,
    verbose = 1,
    validation_data=test_gen,
    callbacks = [es]
    )
model_InceptionV3.save('model_InceptionV3.h5') 

Epoch 1/20
172/172 [==============================] - 257s 1s/step - loss: 3.0564 - categorical_accuracy: 0.7988 - val_loss: 0.2983 - val_categorical_accuracy: 0.9034
Epoch 2/20
172/172 [==============================] - 234s 1s/step - loss: 0.2410 - categorical_accuracy: 0.9197 - val_loss: 0.2362 - val_categorical_accuracy: 0.9204
Epoch 3/20
172/172 [==============================] - 239s 1s/step - loss: 0.1798 - categorical_accuracy: 0.9394 - val_loss: 0.1862 - val_categorical_accuracy: 0.9375
Epoch 4/20
172/172 [==============================] - 242s 1s/step - loss: 0.1344 - categorical_accuracy: 0.9545 - val_loss: 0.1797 - val_categorical_accuracy: 0.9401
Epoch 5/20
172/172 [==============================] - 235s 1s/step - loss: 0.1185 - categorical_accuracy: 0.9607 - val_loss: 0.1875 - val_categorical_accuracy: 0.9408
Epoch 6/20
172/172 [==============================] - 238s 1s/step - loss: 0.1022 - categorical_accuracy: 0.9650 - val_loss: 0.1726 - val_categorical_accuracy: 0.943

In [ ]:
model_InceptionV3 = tf.keras.models.load_model('model_InceptionV3.h5')
_, acc = model_InceptionV3.evaluate(valid_gen, verbose=0)
print('> %.3f' % (acc * 100.0))

NameError: ignored

## ResNet50

In [ ]:
shape = (224, 224)
train_gen = datagen.flow_from_directory(train_dir, target_size = shape, batch_size = batch_size, class_mode = 'categorical', color_mode = 'rgb', shuffle=True)
valid_gen = test_datagen.flow_from_directory(validation_dir, target_size = shape, batch_size = batch_size, class_mode = 'categorical', color_mode = 'rgb', shuffle=False)
test_gen = test_datagen.flow_from_directory(test_dir, target_size = shape, batch_size = batch_size, class_mode = 'categorical', color_mode = 'rgb', shuffle=False)

Found 43831 images belonging to 13 classes.
Found 5474 images belonging to 13 classes.
Found 5489 images belonging to 13 classes.


In [ ]:
from tensorflow.keras.applications import ResNet50

In [ ]:
base_model = ResNet50(input_shape=(224, 224,3), include_top=False, weights="imagenet")
# Freeze convolutional layers
for layer in base_model.layers:
    layer.trainable = False    

# Establish new fully connected block
x = base_model.output
x = Flatten()(x)  # flatten from convolution tensor output  
x = Dense(1000, activation='relu')(x) # number of layers and units are hyperparameters, as usual
predictions = Dense(13, activation='softmax')(x) # should match # of classes predicted

# this is the model we will train
model_ResNet50 = Model(inputs=base_model.input, outputs=predictions)
model_ResNet50.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

94781440/94765736 [==============================] - 1s 0us/step


In [ ]:
epochs = 5
if os.path.exists('model_ResNet50.h5'):
  model_ResNet50 = tf.keras.models.load_model('model_ResNet50.h5')
  print("Model loaded")

history = model_ResNet50.fit(
    train_gen, 
    epochs=epochs,
    verbose = 1,
    validation_data=test_gen,
    callbacks = [es]
    )
model_ResNet50.save('model_ResNet50.h5') 

Epoch 1/5
172/172 [==============================] - 498s 3s/step - loss: 4.3471 - categorical_accuracy: 0.4595 - val_loss: 1.7025 - val_categorical_accuracy: 0.4966
Epoch 2/5
172/172 [==============================] - 488s 3s/step - loss: 1.4501 - categorical_accuracy: 0.5709 - val_loss: 1.3835 - val_categorical_accuracy: 0.5826
Epoch 3/5
172/172 [==============================] - 488s 3s/step - loss: 1.3246 - categorical_accuracy: 0.6035 - val_loss: 1.3047 - val_categorical_accuracy: 0.6351
Epoch 4/5
172/172 [==============================] - 490s 3s/step - loss: 1.2518 - categorical_accuracy: 0.6189 - val_loss: 1.2124 - val_categorical_accuracy: 0.6247
Epoch 5/5
172/172 [==============================] - 491s 3s/step - loss: 1.1757 - categorical_accuracy: 0.6318 - val_loss: 1.1554 - val_categorical_accuracy: 0.6253


In [ ]:
model_ResNet50 = tf.keras.models.load_model('model_ResNet50.h5')
_, acc = model_ResNet50.evaluate(valid_gen, verbose=0)
print('> %.3f' % (acc * 100.0))

> 63.756


## EfficientNet

In [ ]:
shape = (224, 224)
train_gen = datagen.flow_from_directory(train_dir, target_size = shape, batch_size = batch_size, class_mode = 'categorical', color_mode = 'rgb', shuffle=True)
valid_gen = test_datagen.flow_from_directory(validation_dir, target_size = shape, batch_size = batch_size, class_mode = 'categorical', color_mode = 'rgb', shuffle=False)
test_gen = test_datagen.flow_from_directory(test_dir, target_size = shape, batch_size = batch_size, class_mode = 'categorical', color_mode = 'rgb', shuffle=False)

Found 43831 images belonging to 13 classes.
Found 5474 images belonging to 13 classes.
Found 5489 images belonging to 13 classes.


In [ ]:
import tensorflow.keras.applications.efficientnet
base_model = efficientnet.keras.EfficientNetB0(input_shape = (224, 224, 3), include_top = False, weights = 'imagenet')
# Freeze convolutional layers
for layer in base_model.layers:
    layer.trainable = False    

# Establish new fully connected block
x = base_model.output
x = Flatten()(x)  # flatten from convolution tensor output  
x = Dense(1000, activation='relu')(x) # number of layers and units are hyperparameters, as usual
predictions = Dense(13, activation='softmax')(x) # should match # of classes predicted

# this is the model we will train
model_EfficientNetB0 = Model(inputs=base_model.input, outputs=predictions)
model_EfficientNetB0.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

AttributeError: ignored

In [ ]:
epochs = 5
if os.path.exists('model__EfficientNetB0.h5'):
  model_EfficientNetB0 = tf.keras.models.load_model('model__EfficientNetB0.h5')
  print("Model loaded")

history = model_EfficientNetB0.fit(
    train_gen, 
    epochs=epochs,
    verbose = 1,
    validation_data=test_gen
    callbacks = [es]
    )
model_EfficientNetB0.save('model_EfficientNet.h5') 

In [ ]:
model_EfficientNetB0 = tf.keras.models.load_model('model_EfficientNet.h5')
_, acc = model_EfficientNetB0.evaluate(valid_gen, verbose=0)
print('> %.3f' % (acc * 100.0))

## Xception

In [ ]:
from tensorflow.keras.applications.xception import Xception

shape = (224, 224)
train_gen = datagen.flow_from_directory(train_dir, target_size = shape, batch_size = batch_size, class_mode = 'categorical', color_mode = 'rgb', shuffle=True)
valid_gen = test_datagen.flow_from_directory(validation_dir, target_size = shape, batch_size = batch_size, class_mode = 'categorical', color_mode = 'rgb', shuffle=False)
test_gen = test_datagen.flow_from_directory(test_dir, target_size = shape, batch_size = batch_size, class_mode = 'categorical', color_mode = 'rgb', shuffle=False)

base_model = Xception(input_shape = (224, 224, 3), include_top = False, weights = 'imagenet')
# Freeze convolutional layers
for layer in base_model.layers:
    layer.trainable = False    

# Establish new fully connected block
x = base_model.output
x = Flatten()(x)  # flatten from convolution tensor output  
x = Dense(1000, activation='relu')(x) # number of layers and units are hyperparameters, as usual
predictions = Dense(13, activation='softmax')(x) # should match # of classes predicted

# this is the model we will train
model_Xception = Model(inputs=base_model.input, outputs=predictions)
model_Xception.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

Found 43831 images belonging to 13 classes.
Found 5474 images belonging to 13 classes.
Found 5489 images belonging to 13 classes.
83697664/83683744 [==============================] - 0s 0us/step


In [ ]:
epochs = 10
if os.path.exists('model_Xception.h5'):
    model_Xception = tf.keras.models.load_model('model_Xception.h5')
    print("Model loaded")
    history = model_Xception.fit(
    train_gen, 
    epochs=epochs,
    verbose = 1,
    validation_data=test_gen,
    callbacks = [es]
    )
    model.save('model_Xception.h5') 
else:
    history = model_Xception.fit(
        train_gen, 
        epochs=epochs,
        verbose = 1,
        validation_data=test_gen,
        callbacks = [es]
        )
    model_Xception.save('model_Xception.h5') 

Epoch 1/10
172/172 [==============================] - 556s 3s/step - loss: 1.6884 - categorical_accuracy: 0.8525 - val_loss: 0.2158 - val_categorical_accuracy: 0.9282
Epoch 2/10
172/172 [==============================] - 543s 3s/step - loss: 0.1518 - categorical_accuracy: 0.9503 - val_loss: 0.1285 - val_categorical_accuracy: 0.9581
Epoch 3/10
172/172 [==============================] - 528s 3s/step - loss: 0.0914 - categorical_accuracy: 0.9685 - val_loss: 0.1091 - val_categorical_accuracy: 0.9650
Epoch 4/10
172/172 [==============================] - 527s 3s/step - loss: 0.0717 - categorical_accuracy: 0.9759 - val_loss: 0.1193 - val_categorical_accuracy: 0.9597
Epoch 5/10
172/172 [==============================] - 535s 3s/step - loss: 0.0783 - categorical_accuracy: 0.9745 - val_loss: 0.1227 - val_categorical_accuracy: 0.9627
Epoch 6/10
172/172 [==============================] - 534s 3s/step - loss: 0.0558 - categorical_accuracy: 0.9813 - val_loss: 0.1024 - val_categorical_accuracy: 0.968

In [ ]:
model_Xception = tf.keras.models.load_model('model_Xception.h5')
_, acc = model_Xception.evaluate(valid_gen, verbose=0)
print('> %.3f' % (acc * 100.0))

> 96.712


## DenseNet

In [ ]:
from tensorflow.keras.applications.densenet import DenseNet201

shape = (224, 224)
train_gen = datagen.flow_from_directory(train_dir, target_size = shape, batch_size = batch_size, class_mode = 'categorical', color_mode = 'rgb', shuffle=True)
valid_gen = test_datagen.flow_from_directory(validation_dir, target_size = shape, batch_size = batch_size, class_mode = 'categorical', color_mode = 'rgb', shuffle=False)
test_gen = test_datagen.flow_from_directory(test_dir, target_size = shape, batch_size = batch_size, class_mode = 'categorical', color_mode = 'rgb', shuffle=False)

base_model = DenseNet201(input_shape = (224, 224, 3), include_top = False, weights = 'imagenet')
# Freeze convolutional layers
for layer in base_model.layers:
    layer.trainable = False    

# Establish new fully connected block
x = base_model.output
x = Flatten()(x)  # flatten from convolution tensor output  
x = Dense(1000, activation='relu')(x) # number of layers and units are hyperparameters, as usual
predictions = Dense(13, activation='softmax')(x) # should match # of classes predicted

# this is the model we will train
model_DenseNet201 = Model(inputs=base_model.input, outputs=predictions)
model_DenseNet201.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model_DenseNet201.summary()

Found 43831 images belonging to 13 classes.
Found 5474 images belonging to 13 classes.
Found 5489 images belonging to 13 classes.
Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 zero_padding2d_2 (ZeroPadding2  (None, 230, 230, 3)  0          ['input_2[0][0]']                
 D)                                                                                               
                                                                                                  
 conv1/conv (Conv2D)            (None, 112, 112, 64  9408    

In [ ]:
print(X_train.shape)
print(y_train.shape)
print(predict.shape)

(256, 224, 224, 3)
(256, 13)
(256, 7, 7, 1920)


In [ ]:
print(train_gen.class_indices)

{'B': 0, 'K': 1, 'N': 2, 'P': 3, 'Q': 4, 'R': 5, '_': 6, 'b': 7, 'k': 8, 'n': 9, 'p': 10, 'q': 11, 'r': 12}


In [ ]:
epochs = 5
if os.path.exists('model_DenseNet201.h5'):
    model_DenseNet201 = tf.keras.models.load_model('model_DenseNet201.h5')
    print("Model loaded")
    history = model_DenseNet201.fit(
    train_gen, 
    epochs=epochs,
    verbose = 1,
    validation_data=test_gen,
    callbacks = [es]
    )
    model_DenseNet201.save('model_DenseNet201.h5') 
else:
    history = model_DenseNet201.fit(
        train_gen, 
        epochs=epochs,
        verbose = 1,
        validation_data=test_gen,
        callbacks = [es]
        )
    model_DenseNet201.save('model_DenseNet201.h5')

Epoch 1/5


KeyboardInterrupt: ignored

In [ ]:
model_DenseNet201 = tf.keras.models.load_model('model_DenseNet201.h5')
_, acc = model_DenseNet201.evaluate(valid_gen, verbose=0)
print('> %.3f' % (acc * 100.0))

> 97.242


## Using classifiers

In [ ]:
import os
os.remove("X_train.csv")
os.remove("y_train.csv")
os.remove("X_test.csv")
os.remove("y_test.csv")

In [ ]:
X_train = np.array(pd.read_csv("X_train.csv"))[1:, 1:]
y_train = np.array(pd.read_csv("y_train.csv"))[1:, 1:]
X_test = np.array(pd.read_csv("X_test.csv"))[1:, 1:]
y_test = np.array(pd.read_csv("y_test.csv"))[1:, 1:]

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
le.fit(['B', 'K', 'N', 'P', 'Q', 'R', '_', 'b', 'k', 'n', 'p', 'q', 'r'])
# le.transform(y_train)
# le.transform(y_test)

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit_transform(X_train)
scaler.fit_transform(X_test)

from sklearn.neural_network import MLPClassifier
######## MLP ########
mlp = MLPClassifier(hidden_layer_sizes = (500, 500, 500, 500, 500))
mlp.fit(X_train, y_train)
y_pred = mlp.predict(X_test)

from sklearn import metrics
print("Train accuracy score =", metrics.accuracy_score(y_train.reshape(len(y_train),), mlp.predict(X_train)))
print("Accuracy score =", metrics.accuracy_score(y_test.reshape(len(y_test),), y_pred))

/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:1109: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:699: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")


Train accuracy score = 0.9899383983572895
Accuracy score = 0.9624635568513119


In [ ]:
from sklearn.ensemble import RandomForestClassifier
######## RF ########
rf = RandomForestClassifier(n_estimators = 300, min_samples_split = 30)
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)

from sklearn import metrics
print("Train accuracy score =", metrics.accuracy_score(y_train.reshape(len(y_train),), rf.predict(X_train)))
print("Accuracy score =", metrics.accuracy_score(y_test.reshape(len(y_test),), y_pred))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  after removing the cwd from sys.path.


Train accuracy score = 0.9529545973077801
Accuracy score = 0.8438411078717201
